In [11]:
import requests, pandas, getpass

api = 'https://api.earthref.org/v1/MagIC/{}'
username = input()
password = getpass.getpass()

### Create a Private Contribution and delete it
If no errors are reported, this cell can be repeated without any side effects. If noone has also made a private contribution between repeated executions, the same contribution ID should be reused.

In [4]:
create_response = requests.post(api.format('private'), auth=(username, password))
print(create_response.request.method, create_response.request.url)
if (create_response.status_code == 200):
    contribution_id = create_response.json()['id']
    print('Created private contribution with ID', contribution_id, '\n')

    delete_response = requests.delete(api.format('private'), params={'id': contribution_id}, auth=(username, password))
    print(delete_response.request.method, delete_response.request.url)
    if (delete_response.status_code == 200):
        print('Deleted private contribution with ID', contribution_id, '\n')
    else:
        print('Delete Private Contribution Error:', delete_response.json()['errors'][0]['message'], '\n')
else:
    print('Create Private Contribution Error:', create_response.json()['errors'][0]['message'], '\n')

POST http://localhost:3200/v1/MagIC/private
Created private contribution with ID 19235 

DELETE http://localhost:3200/v1/MagIC/private?id=19235
Deleted private contribution with ID 19235 



### Validate a Private Contribution and mark it as valid if there are no errors
The contribution ID should be in your private workspace or it will not be found.

In [15]:
contribution_id = 19288
response = requests.put(api.format('private/validate'), params={'id': contribution_id}, auth=(username, password))
print(response.request.method, response.request.url)
if (response.status_code == 200):
    validation_results = response.json()['validation']
    print('Validated contribution with ID', contribution_id, '\n', validation_results)
elif (response.status_code == 204):
    print('A private contribution with ID', contribution_id, 'could not be found in your private workspace for validation\n')
else:
    print('Error Validating a Private Contribution:', response.json(), '\n')

PUT https://api.earthref.org/v1/MagIC/private/validate?id=19288
Validated contribution with ID 19288 :
 {'errors': [], 'warnings': []}


### Download a Public Contribution and create a Private Contribution to upload it to

In [ ]:
contribution_id = 16901

response = requests.get(api.format('data'), params={'id': contribution_id})
print(response.request.method, response.request.url)
if (response.status_code == 200):
    contribution_file = 'downloads/magic_contribution_{}.txt'.format(contribution_id)
    open(contribution_file, 'w').write(response.text)
    print('Retrieved contribution data with ID', contribution_id, '\n')
    
    create_response = requests.post(api.format('private'), auth=(username, password))
    print(create_response.request.method, create_response.request.url)
    if (create_response.status_code == 200):
        new_contribution_id = create_response.json()['id']
        print('Created private contribution with ID', new_contribution_id, '\n')
        
        with open(contribution_file, 'rb') as f:
            upload_response = requests.put(api.format('private'), 
                params={'id': new_contribution_id}, 
                auth=(username, password), 
                headers={'Content-Type': 'text/plain'}, 
                data=f
            )

        print(upload_response.request.method, upload_response.request.url)
        if (upload_response.status_code == 200):
            print('Uploaded a text file to private contribution with ID', contribution_id, '\n')
        else:
            print('Upload Private Contribution Error:', upload_response.json()['errors'][0]['message'], '\n')
    else:
        print('Create Private Contribution Error:', create_response.json()['errors'][0]['message'], '\n')
else:
    print('Retrieve Public Contribution Error:', response.json()['errors'][0]['message'], '\n')